In [22]:
#Import Libraries
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)
import pymongo
import json
import re
import string
import nltk
import emoji

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob

In [2]:
#Set up variables to access MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
# Database Name
database = client["collegedb"]
# Collection Name
collection = database["breakingNews"]

In [3]:
#Show connection
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'collegedb'), 'breakingNews')

In [4]:
breakingNews = pd.DataFrame.from_records(list(collection.find()))

In [6]:
breakingNews['tweetDate']=pd.to_datetime(breakingNews['tweetcreatedts']).dt.date
breakingNews['tweetDate'] = breakingNews['tweetDate'].astype("datetime64[ns]")
#breakingNews['tweetDate'].groupby(breakingNews["tweetDate"].dt.date).count().plot(kind="bar")

In [18]:
def remove_duplicates(sentence):
    words = sentence.split(" ")
    result = []
    for word in words:
        if word not in result:
            result.append(word)
    return " ".join(result)

def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove urls
    text = re.sub(r'http\S+', '', text)
    # Remove mentions and hashtags
    text = re.sub(r'@\w+|\#\w+', '', text)
    # Remove emojis
    text = (emoji.replace_emoji(text, ''))
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove extra chars
    text = re.sub(r"[^\w\s]+", "", text)
    # Remove Duplicates
    text = remove_duplicates(text)
    # Tokenize text into words
    words = nltk.word_tokenize(text)
    # Lemmatize words
    lem = WordNetLemmatizer()
    words = [lem.lemmatize(word) for word in words]
    # Remove stop words
    words = [word for word in words if word not in stpwrd]
    # Join words back into text
    text = ' '.join(words)
    return text

def count_words(s):
    return len(s.split())

In [10]:
breakingNews.shape

(4946365, 4)

In [8]:
breakingNewsSamp = breakingNews.groupby('tweetDate').apply(lambda x: x.sample(frac=.1))

In [10]:
breakingNewsSamp.shape

(494634, 4)

In [28]:
#Add commonly seen words that are not info related.
new_stopwords = ["news", "ha", "u", "via", "amp", "link", "wa"]
stpwrd = nltk.corpus.stopwords.words('english')
stpwrd.extend(new_stopwords)

breakingNewsSamp['clean_text'] = breakingNewsSamp['text'].apply(clean_text)
breakingNewsSamp['countWords'] = breakingNewsSamp['clean_text'].apply(count_words)
breakingNewsSamp = breakingNewsSamp[breakingNewsSamp['countWords']>5]
print(breakingNewsSamp.shape)

(413849, 7)


In [ ]:
breakingNewsSamp.head(5)

from collections import Counter
Counter(" ".join(breakingNewsSamp['clean_text']).split()).most_common(100)

In [33]:
from nltk.sentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

breakingNewsSamp['polarity'] = breakingNewsSamp['clean_text'].apply(lambda x: analyzer.polarity_scores(x))
breakingNewsSamp['compound'] = [analyzer.polarity_scores(x)['compound'] for x in breakingNewsSamp['clean_text']]
breakingNewsSamp['neg'] = [analyzer.polarity_scores(x)['neg'] for x in breakingNewsSamp['clean_text']]
breakingNewsSamp['neu'] = [analyzer.polarity_scores(x)['neu'] for x in breakingNewsSamp['clean_text']]
breakingNewsSamp['pos'] = [analyzer.polarity_scores(x)['pos'] for x in breakingNewsSamp['clean_text']]

In [35]:
breakingNewsSamp.head(30)

_id                                                                                                                                                                                                                                                                                                                   text       tweetcreatedts  tweetDate                                                                                                                                                      clean_text                                                        polarity  countWords  compound    neg    neu    pos
tweetDate                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
2022-08-02 2251651  644d2cfd1fed118138fca732                      Pakistan's annual inflation for the month of July has reached 24.93. \n#Pakistan #pakistanopinion #LatestNews #pakistanlatestnews #pakistanopinion #PakistanNews #pakistannewstoday #inflation #inflation2022 #inflationinpakistan #highinflation #pakistanunderinflation https://t.co/wJ3ePPiyKx  2022-08-02 10:46:51 2022-08-02                                                                                                               pakistan annual inflation month july reached 2493    {'neg': 0.0, 'neu': 0.811, 'pos': 0.189, 'compound': 0.1027}           7    0.1027  0.000  0.811  0.189
           2251877  644d2cfd1fed118138fca814                                                            Texas Children&amp;#039;s Pavilion for Women Celebrates the Completion of Phase One of its Multi-Year Expansion Plan\nhttps://t.co/UQXhsrmmcK\n #business #industrynews #News #IndustryNews #LatestNews #LatestIndustryNews #PRNews https://t.co/WzG1GOgXZP  2022-08-02 23:00:30 2022-08-02                                                                      texas childrenamps pavilion woman celebrates completion phase one multiyear expansion plan      {'neg': 0.0, 'neu': 0.73, 'pos': 0.27, 'compound': 0.5719}          11    0.5719  0.000  0.730  0.270
           2251693  644d2cfd1fed118138fca75c                                                                                                               Caribbean leaders extend wishes on the Emancipation Day 2022\n\nhttps://t.co/SMufyFfevy\n\n#Caribbean #CaribbeanRegion #EmancipationDay #EmancipationDay2022 #News #LatestNews #LatestUpdate #Writeups24  2022-08-02 12:30:00 2022-08-02                                                                                                              caribbean leader extend wish emancipation day 2022    {'neg': 0.0, 'neu': 0.532, 'pos': 0.468, 'compound': 0.5267}           7    0.5267  0.000  0.532  0.468
           2251810  644d2cfd1fed118138fca7d1                                  Become a member of our LinkedIn group to receive the latest healthcare, telehealth, and telemedicine-related news and articles.\n\nhttps://t.co/03Hr3FzzAn\n\n#LinkedIn #Linkedingroup #telehealth #Telemedicine #healthcare #groupchat #articles #latestnews https://t.co/mS27zmIbnh  2022-08-02 16:30:00 2022-08-02                                                                   become member linkedin group receive latest healthcare telehealth telemedicinerelated article           {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}          10    0.0000  0.000  1.000  0.000
           2251522  644d2cfd1fed118138fca6b1                                                               'Inc

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers

max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
tweets = pad_sequences(sequences, maxlen=max_len)
print(tweets)

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MongoDBInput") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/college.breakingNews") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/college.breakingNews") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.13:10.1.1") \
    .getOrCreate()

df = spark.read.format("com.mongodb.spark.sql.DefaultSource") \
    .option("uri", "mongodb://localhost:27017/college.breakingNews") \
    .load()